In [ ]:
%alias_magic solidity script -p "/home/jovyan/.solcx/solc-v0.8.0 --ast-json -"

Created `%%solidity` as an alias for `%%script /home/jovyan/.solcx/solc-v0.6.7 --ast-json -`.


In [7]:
%alias_magic ape script -p "/home/jovyan/bin/ape.bin -csyntax"

Created `%%ape` as an alias for `%%script /home/jovyan/bin/ape.bin -csyntax`.


In [ ]:
%alias_magic prolog script -p "/home/jovyan/swipl"

# License Evaluation Contract Representation

In the following notebook the examples of my thesis are accumulated in such a state as to make 

## Natural legal language example

### License and Evaluation Agreement
This is a license agreement for a software evaluation license of `<Product>`, dated `<Date>` between the licensor `<Licensor Name>` and licensee `<Licensee Name>`. It describes the terms for evaluation of a software product and subsequent publication of comments about the evaluation.


* __Article 1.__	The Licensor grants the Licensee a limited license to use and evaluate asset X and grant sublicenses among group Y, for use and evaluation, in exchange for a licensing fee. This limited license lasts from DD.MM.YYYY (inclusive) to DD.MM.YYYY (exclusive). The licensing fee in amount of `<Amount>` has to be paid within 5 days after contract formation (signature).

* __Article 2.__	(optional) The (Sub)Licensee is commissioned to publish comments about the use of the product, in exchange for an evaluation fee. This allows publication of comments about the evaluation of *ASSET*. Furthermore, this requires the publication of comments about the evaluation of the *ASSET*. If comments are not given for approval within the license period, the Licensee is in breach of the agreement.

* __Article 3.__	The (Sub)Licensee must not publish comments of the use and evaluation of the Product without the approval of the Licensor; said approval must be obtained before the publication of comments about the evaluation of *ASSET*. If the Licensee publishes results of the evaluation of the Product without approval from the Licensor, the Licensee has 24 h to remove the material from the time of publication. If the time lapses without a removal of the comments of the evaluation, the license is considered breached. If the comments are removed, the previous license previous state is restored.

* __Article 4.__	The Licensor is required to approve or deny the comments within 5 days. Furthermore, the Licensor is required to license on successful contract formation and has to pay an evaluation fee in amount `<Amount>` of to the licensee on successful publication of commissioned comments.

* __Article 5.__	Events of Breach

This license agreement is considered breached if – 
* The licensor fails to grant a license within 5 days
* The licensee fails to pay the license fee within 5 days
* The license is used outside of the license period
* In case of publication without approval or failure to publish in case of commission, the defect is not remedied within 24 H
* The licensor unduly delays (longer than 5 days) approval or disapproval of submitted comments
* In case of commission, the licensor fails to pay an evaluation fee upon approval of the comments (within 5 days)

* __Article 6.__	This license terminates automatically if the Licensee, Licensor or a sublicensee breaches this license agreement. In case of a breach by the main Licensee, this entails termination of all sublicensees. A breach of the Licensing Terms obliges the Licensee  or Licensor to pay a fee to Licensor or Licensee respectively for Breach of the Licensing Terms.


Coice of Law, Modification & Dispute Resolution
This license evaluation agreement is subject to the laws of <Place>. Any modifications to the agreement are only effective if agreed to and executed by both parties. Parties agree to binding arbitration by <Person/Place/Institution> either via electronic or ordinary means. Any notices and communication under this agreement are to be sent via electronic message or equivalent, depending on the electronic contract system used.
	

`Licensor                                                                                                     Licensee`




`Signature                                                                                                   Signature`


## Solidity

Soliditiy can be compiled, but for testing its functions, it would have to be copied into remix ide. Also it should be noted that for abbrevation purposes, this contract is not necessarily secure or complete from a usability point of view, such as convenience functions and similar might be missing. This is due to the fact that it would otherwise be too long for publication.

In [ ]:
// SPDX-License-Identifier: GPL-3.0
pragma solidity ^0.8.0;

import "contracts/datetime.sol";
import "contracts/Ownable.sol";

/// @title A (software) license evaluation system
/// @author Florian Idelberger
/// @notice A license contract to provide software evaluation licenses and functions to check status
/// @dev require is always used instead of assert so far, to provide an error message and make the code more readable

contract LicenseContract is Ownable {
    uint numLicenses;
    // Constructor
    constructor() {
        // This tracks the number of licenses, here initialized to 0
        numLicenses = 0;
    }

// Events
event LicenseStatus(LicenseContract.License);
event LicenseID(uint licenseID);
event LicenseContractStatus(LicenseContract);
event LicenseFeeStatus(uint256 amount);
event BreachFeeStatus(breachFee bfs);
event SublicenseeStatus(uint numberOfSublicensees);

mapping(uint256 => address) public licensor;
mapping(uint256 => address) public licensee;
mapping(uint256 => address) public arbiter;
mapping(uint256 => License) public licenses;
mapping(uint256 => address[]) public sublicensees;
mapping (address => uint) pendingLicenseFeeWithdrawals;
mapping (address => breachFee) breachFeeStorage;

// This struct ties a brechFee to a license and is necessary
// to track if a breachFee was cleared for withdrawal.
struct breachFee {
    uint256 licenseID;
    bool cleared;
    uint amount;
}

// As far as rules are not set out specifically,
// but only defined imperatively, this struct sets
// out the default rules, in so far without being set,
// the permissions are always false.

struct License {
    //uint256 licenseID; 
    bytes32 workHash; // this might actually not be necessary and could be covered by licenseID but out of scope how this would work it trying to enforce client-side
    uint8 licenseFee;
    uint8 breachFee;
    bool isCommissioned;
    bool publicationIsApproved;
    bool unauthorizedPublication;
    uint timeToRemove;
    uint timeOfNotification;
    bool licenseBreached;
    uint startdate;
    uint enddate;
}

// OnlyBy modifier. Ideally would allow for multiple adresses.
modifier onlyBy(address _account)
    {require(msg.sender == (_account),
          "Sender not authorized.");
        _;}

/// @notice Create a new license and add it to the mapping of licenses.
/// @dev For now licensor is fixed to owner. timeToRemove is assumed to be in days. Wanted to specify a date or timestamp,
// but if anything would have to be a date diff, but to get that would need a date. Is there a difference between built-in + 1 day and addDays in BokkyPooBahsDateTimeLibrary?
/// @param _license passes a license struct.
/// @return _licenseID of the created license.
function newLicense(License memory _license) public onlyOwner returns (uint _licenseID) {
    //require(_license.licenseID != 0, "License ID has to be provided and cannot be 0"); // require license id, license fee, breach fee, startdate and enddate
    require(_license.licenseFee != 0, "License Fee has to be provided and cannot be 0.");
    require(_license.breachFee != 0, "Breach Fee has to be provided and cannot be 0.");
    uint startyear; uint startmonth; uint startday;
    uint endyear; uint endmonth; uint endday;
    (startyear, startmonth, startday) = BokkyPooBahsDateTimeLibrary.timestampToDate(_license.startdate);
    (endyear, endmonth, endday) = BokkyPooBahsDateTimeLibrary.timestampToDate(_license.enddate);
    require(BokkyPooBahsDateTimeLibrary.isValidDate(startyear, startmonth, startday),
        "Seems to be an invalid startdate."
    );
    require(BokkyPooBahsDateTimeLibrary.isValidDate(endyear, endmonth, endday),
        "Seems to be an invalid enddate."
    );
    require(_license.publicationIsApproved == false);
    require(_license.unauthorizedPublication == false);
    require(_license.licenseBreached == false);
    require(_license.timeToRemove > 0, "Time for removal of unauthorized publication has to be greater than 0.");
    LicenseContract.numLicenses++; // starts at 1
    _licenseID = LicenseContract.numLicenses;
     // check if incrementation works
    licensor[_licenseID] = owner();
    licenses[_licenseID] = _license;
    emit LicenseID(_licenseID);
    emit LicenseStatus(_license);
    }

/// @notice 'Sign' the license by assigning a licensee and paying license and breachFee.
/// @dev The successful signing and payment of the fees concludes a valid license.
/// @param _licenseID of the license to be modified, and address of the _licensee to be added
function signLicense(uint _licenseID, address _licensee) public payable onlyBy(msg.sender) {
    require(arbiter[_licenseID] != address(0x0), "No arbiter specified");
    licensee[_licenseID] = _licensee;
    //payable(this.address).send(licenses[_licenseID].licenseFee);
    (bool success, ) = address(this).call{value: licenses[_licenseID].licenseFee}("");
    require(success, "Transfer of license fee failed.");
    pendingLicenseFeeWithdrawals[licensor[_licenseID]] += licenses[_licenseID].licenseFee;
    (bool success_breach, ) = address(this).call{value: licenses[_licenseID].breachFee}("");
    require(success_breach, "Transfer of breach fee failed.");
    breachFeeStorage[licensor[_licenseID]] = breachFee(_licenseID, false, licenses[_licenseID].breachFee);
    emit LicenseStatus(licenses[_licenseID]);
    emit LicenseFeeStatus(pendingLicenseFeeWithdrawals[licensor[_licenseID]]);
    emit BreachFeeStatus(breachFeeStorage[licensor[_licenseID]]);
}

/// @notice Allow withdrawal of license fee.
/// @dev The licensor can withdraw pending license fee(s).
/// @param _licenseID of the relevant license
function withdrawLicenseFee(uint _licenseID) public onlyBy(licensor[_licenseID]) returns (bool success) { // licenseID only used for modifier so maybe could be replaced with msg.sender
        uint amount = pendingLicenseFeeWithdrawals[msg.sender];
        // Remember to zero the pending refund before
        // sending to prevent re-entrancy attacks
        pendingLicenseFeeWithdrawals[msg.sender] = 0;
        (bool success, ) = licensor[_licenseID].call{value: amount}("");
        require(success, "Withdrawal of license fee failed.");
        emit LicenseFeeStatus(pendingLicenseFeeWithdrawals[msg.sender]);
    }

/// @notice This function sets the arbiter
/// @dev Should be set before signing the license.
/// @param _licenseID of the license to be modified and address of the arbiter to be added
function setArbiter(uint _licenseID, address _arbiter) public onlyBy(licensor[_licenseID]) { // onlyOwner
    require(arbiter[_licenseID] == address(0x0), 'Arbiter already set.');
    arbiter[_licenseID] = _arbiter;
    emit LicenseContractStatus(this);
}

/// @notice This adds a sublicensee to an existing license.
/// @dev This takes one address at a time and uses an array mapped to an id, returning number of sublicenses of a license. (in future -  allow adding array of addresses)
/// @param _licenseID of the license to be modified and the address of the arbiter to be added
function addSublicensee(uint _licenseID, address _sublicensee) public onlyBy(licensor[_licenseID]) returns (uint sllenght) {
    require(licensee[_licenseID] != address(0x0), 'License has to be signed before Sublicensees can be added.');
    sublicensees[_licenseID].push(_sublicensee);
    uint sllength = sublicensees[_licenseID].length;
    emit SublicenseeStatus(sllength);
    return sllength; }

/// @notice This commissions comments. It has to be set before signing of the license.
/// @dev TODO - check if this require actually does what it should/include in test // commssion fee left out for now.
//  Would also use withdrawal pattern and also should allow licensor to recover funds if they are not withdranw until a certain point
/// @param _licenseID of the license to be modified
function commissionComments(uint _licenseID) public onlyBy(licensor[_licenseID]) {
    // date before end date
    require(licensee[_licenseID] == address(0x0), 'Unilateral modification is not permitted.');
    //(bool success, ) = address(this).call{value: licenses[_licenseID].commissionFee}("");
    //require(success, "Transfer of commissioning payment failed.");
    licenses[_licenseID].publicationIsApproved = true;
    licenses[_licenseID].isCommissioned = true;
    emit LicenseStatus(licenses[_licenseID]);
    }

/// @notice This grants approval for comments, can be used before or after signing.
/// @dev At present, this is only allowed to be called by the licensor.
/// @param _licenseID of the license to be modified
function grantApproval(uint _licenseID) public onlyBy(licensor[_licenseID]) {
    require(licensee[_licenseID] != address(0x0), 'Does not make sense to grant approval before a signed license.');
    require(block.timestamp > licenses[_licenseID].startdate && block.timestamp < licenses[_licenseID].enddate, "Outside of licensing period.");
    require(licenses[_licenseID].publicationIsApproved == false, "Publication was already approved");
    require(licenses[_licenseID].unauthorizedPublication == false, "Unauthorized publication, no ex-post approval allowed.");
    licenses[_licenseID].publicationIsApproved = true; 
    emit LicenseStatus(licenses[_licenseID]);
    }

/// @notice This sets unauthorizedPublication if an unauthorized publication was noticed.
/// @dev 
/// @param _licenseID of the license 
function setUnauthorizedPublication(uint _licenseID) public onlyBy(licensor[_licenseID]) returns (uint timeOfNotification) {
    require(block.timestamp < licenses[_licenseID].enddate, "License period ended.");
    require(licensee[_licenseID] != address(0x0), 'Does not make sense to have an unauthorized publication before a signed license.');
    require(licenses[_licenseID].unauthorizedPublication == false, "Unauthorized publication already set.");
    require(licenses[_licenseID].publicationIsApproved == false, "Publication is authorized.");
    licenses[_licenseID].unauthorizedPublication = true;
    if (licenses[_licenseID].timeToRemove == 0) {
        licenses[_licenseID].timeToRemove = 24 hours;
    }
    licenses[_licenseID].timeOfNotification = block.timestamp;
    return licenses[_licenseID].timeOfNotification;
}

/// @notice This function clears license states after a removal of unauthorized publication during the removal period.
/// @dev Right now this is only available for the arbiter, but this is not ideal
/// @param _licenseID of the license to be modified
function declareRemoved(uint _licenseID) public onlyBy(arbiter[_licenseID]) {
    require(block.timestamp > licenses[_licenseID].startdate && block.timestamp < licenses[_licenseID].enddate, "Outside of licensing period."); // or not necessary here?
    require(licensee[_licenseID] != address(0x0), 'Successful removal can only be declared on a signed license.');
    require(licenses[_licenseID].unauthorizedPublication == true, "No unauthorized publication recorded.");
    require(block.timestamp < (licenses[_licenseID].timeOfNotification + licenses[_licenseID].timeToRemove), "Removal period lapsed.");
    licenses[_licenseID].timeToRemove = 0;
    licenses[_licenseID].unauthorizedPublication = false;
    emit LicenseStatus(licenses[_licenseID]);
}

/// @notice This declares a breach, and then clears breach fee for withdrawal
/// @dev 
/// @param _licenseID of the license to be modified
function declareBreach(uint _licenseID) public onlyBy(arbiter[_licenseID]) returns (bool success) {
    require(licensee[_licenseID] != address(0x0), 'Does not make sense to declare a breach on an unsigned license.');
    //require(licenses[_licenseID].unauthorizedPublication); // mmmh this would mean only possibility for breach is unauthorized publication
    require(block.timestamp > licenses[_licenseID].startdate && block.timestamp < licenses[_licenseID].enddate, "Outside of licensing period."); // check for the license period
    require(licenses[_licenseID].timeOfNotification + licenses[_licenseID].timeToRemove <= block.timestamp, "The removal period has not yet lapsed.");
    licenses[_licenseID].licenseBreached = true;
    breachFeeStorage[licensor[_licenseID]].cleared = true;
    sublicensees[_licenseID] = [address(0x0)];
    licensee[_licenseID] = address(0x0);
    emit LicenseStatus(licenses[_licenseID]);
    return true;
         }

/// @notice This allows for withdrawal of the breach fee, assuming it is cleared by the arbiter.
/// @dev 
/// @param _licenseID of the license to be modified
function withdrawBreachFee(uint _licenseID) public onlyBy(licensor[_licenseID])  {
        require(breachFeeStorage[msg.sender].cleared == true, "This breach fee has not been cleared and thus cannot be withdrawn, likely there is no breach.");
        uint amount = breachFeeStorage[msg.sender].amount;
        // Remember to zero the pending refund before
        // sending to prevent re-entrancy attacks
        breachFeeStorage[msg.sender].amount = 0;
        (bool success, ) = licensor[_licenseID].call{value: amount}(""); 
        require(success, "Transfer of breach fee failed.");
    }

/// @notice This allows to 'withdraw' (return) the breach fee if the license was concluded properly.
/// @dev 
/// @param _licenseID of the license that ended
function returnBreachFee(uint _licenseID) public onlyBy(licensee[_licenseID])  {
        require(licensee[_licenseID] != address(0x0), "There is no signed license.");
        require(block.timestamp > licenses[_licenseID].enddate, "The license is still active, the breach fee cannot be returned yet.");
        require(licenses[_licenseID].licenseBreached == false, "License is marked as breached.");
        require(breachFeeStorage[msg.sender].cleared == false, "There was a breach, cannot be withdrawn.");
        uint amount = breachFeeStorage[licensor[_licenseID]].amount; 
        // Remember to zero the pending refund before
        // sending to prevent re-entrancy attacks
        breachFeeStorage[licensor[_licenseID]].amount = 0;
        (bool success, ) = licensee[_licenseID].call{value: amount}("");
        require(success, "Transfer of breach fee failed.");
    }

/// @notice This checks whether the license is active based on date and other data
/// @dev Here  as this is just a passive function and to try another pattern, if/else etc are used instad of require/assert
/// @param _licenseID of the license to be modified
function licenseActivationStatus(uint _licenseID) public returns (bool status) {
        if (licensor[_licenseID] == address(0x0) ) { // || licenses[_licenseID].licenseID != 0
               status = false;
        } else if (licensee[_licenseID] == address(0x0)) {
            status == false;
        } else if (licenses[_licenseID].licenseBreached == true || block.timestamp > licenses[_licenseID].enddate || block.timestamp < licenses[_licenseID].startdate ) {
            status = false;
        } else if (licenses[_licenseID].timeOfNotification != 0 && block.timestamp > (licenses[_licenseID].timeOfNotification + licenses[_licenseID].timeToRemove) ) {
            declareBreach(_licenseID);
        } else {
            status = true;
        }
        emit LicenseStatus(licenses[_licenseID]);
        return status;
    }

    // for receiving
    event Received(address, uint);
    receive() external payable {
        emit Received(msg.sender, msg.value);
    }
}


JSON AST:


======= <stdin> =======
{
  "attributes":
  {
    "absolutePath": "<stdin>",
    "exportedSymbols":
    {
      "LicenseContract":
      [
        283
      ]
    }
  },
  "children":
  [
    {
      "attributes":
      {
        "literals":
        [
          "solidity",
          "^",
          "0.6",
          ".7"
        ]
      },
      "id": 1,
      "name": "PragmaDirective",
      "src": "0:23:0"
    },
    {
      "attributes":
      {
        "literals":
        [
          "experimental",
          "ABIEncoderV2"
        ]
      },
      "id": 2,
      "name": "PragmaDirective",
      "src": "24:33:0"
    },
    {
      "attributes":
      {
        "abstract": false,
        "baseContracts":
        [
          null
        ],
        "contractDependencies":
        [
          null
        ],
        "contractKind": "contract",
        "documentation": null,
        "fullyImplemented": true,
        "linearizedBaseContracts":
        [
          283
        ]

## Logic-based / Prolog

In [ ]:
%%prolog
% Preamble
person(lorna). %licensor
person(lisa). %licenscee
person(sunny). % sublicensee
asset(work). % add asset
 
% defines the starting conditions of the license
licensor(lorna).
licensee(lisa).
uses(license).
made_offer(lorna, lisa).
accepts_offer(lisa, lorna).
paid_license_fee(lisa, lorna, fee).
approves_comments(lorna, lisa).
 
% uncomment to define that comments are submitted
% submitted_comments(LI).
commissions(lorna, lisa). % (optional) 
% sets the licensing date
startdate(date(2021, 2, 25)).
enddate(date(2021, 3, 25)).
 
published_comments(lisa).% lisa published comments
% uncomment to set comment date
% comment_date(date(2021, 12, 24))
% submitted_comments(LI)
% submission_date(date(2021, 12, 22))
 
% Article 1 (license fee set to paid in beginning)
call_licenses(LO, LI) :-
    licenses(LO, LI, _, _).  
licenses(X, Y, A, D) :- 
    asset(A),
    person(X),
    licensor(X),
    person(Y),
    licensee(Y),
    made_offer(X, Y),
    accepts_offer(Y, X),
    paid_license_fee(Y, X, _),
    startdate(_),
    enddate(_),
    current_date(D).
 
% if there is a license for licensee, they may sublicense
may_sublicense(Y) :-
    licenses(_, Y, _, _).
 
% may publish if comments are approved
may_publish(Y) :-
    licenses(X, Y, _, _),
    approves_comments(X, Y).
 
% calculates the license term between startdate and enddate
license_term(S, E, R) :- % checks license term from date S to E, with duration R
    startdate(S),
    from_date(S, SY, SM, SD),
    enddate(E),
    from_date(E, EY, EM, ED),
    RY is EY - SY,
    RM is EM - SM,
    RD is ED - SD,
    % R license term in difference between the two dates
    R = date(RY, RM, RD).
 
% an example license term. (kept here because of warnings if same predicates are kept apart))
license_term(date(2021, 2, 25), date(2021, 3, 25), _).
 
% Article 2
% obligation to publish if there is a commission
must_publish(Y) :- % licensee must publish if ...
    licenses(X, Y, _, _), % ... there is a license between X and Y
    approves_comments(X,Y), % ... X approves of the comments Y made
    commissions(X,Y). % ... and X commissioned Y to comment.
 
% licensor must pay commission fee if required to publish
commission_fee(_, Y) :- % pay commission fee
    must_publish(Y).
 
% Art 3
%forbidden to publish if not approved or not licensed
call_forbidden_publish(Y) :- 
    (not(approves_comments(_, Y))) ; (not(licenses(_, Y, _, _))).
call_forbidden_publish(Y) :- 
    startdate(S),
    enddate(E),
    forbidden_publish(Y, S, E).
% forbidden to publish if outside of license term
forbidden_publish(Y, S, E) :- 
    licenses(_, Y),
    current_date(D), from_date(D, CY, CM, _),
    write(CM), nl,
    write(D), nl,
    startdate(S),
    from_date(S, SY, SM, _),
    write(SM), nl,
    enddate(E),
    from_date(E, EY, EM, _),
    write(EM), nl,
	    % this only checks year and month for simplification
	    ((SY > CY; CY > EY) ; ((SY is CY; CY is EY), (SM > CM; CM > EM)) ). 
	 
	% obligations to remove comments if removal was requested.
	must_remove_comments(Y, T, D, H) :-
	    licensor(LO), requested_removal(LO, Y, T, D, H).
	 
	approved_comments(lorna).
	requested_removal(LO, LI, T, D, H) :- % check who requests? % requested at time T, with day D and hours H
	    licensor(LO),
	    licensee(LI),
	    published_comments(LI),
	   approved_comments(LO2),
	    nonvar(LO2),
	    current_time(T), from_datetime(T, _, _, D, H, _, _).
	 
	% Article 4.
	license_date(date(2021, 5, 10)).
	approval_overdue(LI, LO) :-
	    commissions(LO, LI),
	    current_date(D), from_date(D, _CY, _CM, CD),
	    license_date(D2), from_date(D2, _LY, _LM, LD),
	    CD >= LD + 5.
	    
	 
	pay_eval_fee(LO, _AM) :-
	    licenses(LO, LI, _A, D),
	    commissions(LO, LI),
	    published_comments(LI),
	    current_date(D), from_date(D, _CY, _CM, _),
	    comment_date(DP), from_date(DP, _PY, _PM, _),
	    write('Paid evaluation fee.'), nl.
	 
	% Article 5.
	breach(_) :-
	    uses(_),
	    current_date(CD), from_date(CD, CY, CM, _),
	    startdate(SD), from_date(SD, SY, SM, _),
	    enddate(ED), from_date(ED, EY, EM, _),
	    ((SY > CY; CY > EY) ; ((SY is CY; CY is EY), (SM > CM; CM > EM)) ). 
	% comments are not remvoed
	breach(Y) :-
	    current_time(CT),
	    must_remove_comments(Y, _, DS, HS), % timeset, day set, hour set
	    from_datetime(CT, _, _, CD, CH, _, _),
	    (CH > HS+24; CD > DS). % does not keep state so non-functional...
	% failure to publish
	breach(LI) :-
	    commissions(_, LI),
	    not(published_comments(LI)),
	    startdate(SD), from_date(SD, SY, SM, _),
	    enddate(ED), from_date(ED, EY, EM, _),
	    ((SY > _CY; _CY > EY) ; ((SY is _CY; _CY is EY), (SM > _CM; _CM > EM))). 
	% licensor does not approve comments within 5 days
	breach(LO) :-
	    call_licenses(LO, LI),
	    submitted_comments(LI),
	    submission_date(SUD), from_date(SUD, SUY, SUM, SUD),
	    current_date(D), from_date(D, CY, _CM, CD),
	    not(published_comments(LI)),
	    ((CD > SUD + 5) ; (CY > SUY) ; (CY > SUM)).
	        
	% 5 days after comments no eval fee
	breach(LO) :-
	    commissions(LO, LI),
	    not(pay_eval_fee(LO, _)),
	    published_comments(LI),
	    current_date(D), from_date(D, _CY, _CM, CD),
	    comment_date(D2), from_date(D2, _COY, _COM, COD),
	    CD >= COD + 5.
	 
	% Article 6.
	terminates(LO, LI) :-
	    call_licenses(LO, LI),
	    breach(_).
	terminates_sublicenses(_) :-
	    terminates(_, _).
	pay_breach_fee(LI, LO) :- % in case of breach, pay breach fee to licensor
	    (breach(LI) ; breach(LO)),
	    write('Breach fee paid.'), nl.
	 
	%% utility procedures
	% This gets current date or time
	current_date(D) :-
	    get_time(X), stamp_date_time(X, Y, local), date_time_value('date', Y, D).
	current_time(Y) :-
	    get_time(X), stamp_date_time(X, Y, local).
	 
	% gets components of date and datetime respectively
	from_date(Date, Year, Month, Day) :-
	    arg(1, Date, Year),
	    arg(2, Date, Month),
	    arg(3, Date, Day).
	from_datetime(DateTime, Year, Month, Day, Hours, Mins, Secs) :-
	    arg(1, DateTime, Year),
	    arg(2, DateTime, Month),
	    arg(3, DateTime, Day),
	    arg(4, DateTime, Hours),
	    arg(5, DateTime, Mins),
	    arg(6, DateTime, Secs).


The license example was given above. For querying the example, please use the field below.

In [ ]:
?- must_publish(eve).
?- get_time(X), stamp_date_time(X, Y, local).

## ACE / Attempto

Below is an exexcution of the ACE example, with the Ace Parsing Enginge (APE). It is currently read from a file and not from a notebook cell, but the file can be modified and also output of ape changed to other formats. < Link to APE options >

In [8]:
%%ape
The Licensor is a person, and the Licensee is a person and every n:sublicensee is a person. A n:license-fee is an amount. The asset is a n:computer-program.

/* Article 1 */ 
The Licensor grants the Licensee a limited license. The Licensee pays a n:license-fee to the Licensor. Every license covers at least one asset.
The license allows to use the asset and the license allows to evaluate the asset. The license allows to v:sublicense the asset to the students. Every license has a n:license-duration from Startdate to Enddate.

/* Art 2 */ /* check publish comments */
The Licensor may give a n:commission-to-publish-comments to the Licensee. If the Licensee has a n:commission-to-publish-comments then the Licensee is commissioned. If the Licensee is commissioned then the publication of at least one comment is permitted.
If the Licensee is commissioned then the Licensor must pay an n:evaluation-fee to the Licensee.
If the Licensee is commissioned then the Licensee must publish at least one comment.

/* Art 3 */ /* maybe rephrase a:commission-to-publish   */
The Licensee and a n:sublicensee may not publish at least one comment that is not approved by the Licensor. 
The approval of at least one comment needs to be given before the publication. If the Licensee publishes at least one comment without the approval of the Licensor then the Licensee must remove the comment within 24 h.

/* Art 4 */
If the Licensee v:submits-comments to the Licensor then the Licensor should approve at least one comment within 5 days. The Licensor can deny the comments within 5 days.
If the comments are public and commissioned then the Licensor has to pay an n:evaluation-fee.

/* Art 5 */
If the Licensor fails to grant a license then the license is breached.
If the Licensee fails to pay the n:license-fee within 5 days then the license is breached.
If the license is used outside the n:license-duration then the license is breached.
If the time lapses without the removal of the comments then the license is breached. /* other part specified before */
If the Licensee submits the comments then the Licensor must decide within 5 days.
If the Licensor must pay an n:evaluation-fee and does not pay within 5 days then the license is breached.

/* Art 6 */
If there is a breach then the license terminates. If the license is breached by the Licensee and there is at least one n:sublicensee then the license of all n:sublicensees is terminated. If the Licensee breaches the license then the Licensee has to pay a n:breach-fee to the Licensor. If the Licensor breaches the license then the Licensor must pay a n:breach-fee to the Licensee.


<?xml version="1.0" encoding="UTF-8"?>

<apeResult>
  <duration tokenizer="0.070" parser="0.016" refres="0.001"/>
  <syntax>[[specification,[s,[np,[det,an],[nbar,[n,arbiter]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,licensor]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,licensee]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,sublicensee]]],[vp,[aux,is],[np,[det,a],[nbar,[n,person]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,licensor]]],[vp,[vbar,[vcompl,[v,commissions],[np,[det,a],[nbar,[n,licensee]]]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,'duration-to-remove-comments']]],[vp,[aux,is],[np,[nump,24],[nbar,[n,hours]]]]],'.'],[specification,[s,[np,[det,a],[nbar,[n,'license-fee']]],[vp,[aux,is],[np,[det,an],[nbar,[n,amount]]]]],'.'],[specification,[cond_s,if,[s_coord,[top_s,[topic,[quant,'there is'],[np,[det,a],[nbar,[n,licensor]]]]],[c

In [9]:
/home/jovyan/bin/ape.bin -file license_from_prolog.ace -solo tokens

[[^,'An',arbiter,is,a,person,'.'],[^,'A',n,:,licensor,is,a,person,'.'],[^,'A',licensee,is,a,person,'.'],[^,'A',n,:,sublicensee,is,a,person,'.'],[^,'A',n,:,licensor,commissions,a,licensee,'.'],[^,'A',n,:,'duration-to-remove-comments',is,24,hours,'.'],[^,'A',n,:,'license-fee',is,an,amount,'.'],[^,'If',there,is,a,n,:,licensor,and,there,is,a,licensee,then,a,n,:,'license-fee',is,required,'.'],[^,'If',the,n,:,'license-fee',is,paid,then,the,licensee,receives,a,license,'.'],[^,'If',there,is,a,license,then,the,licensee,can,v,:,sublicense,'.'],[^,'If',there,is,a,license,then,the,licensee,can,comment,'.'],[^,'If',the,licensee,may,comment,then,the,n,:,licensor,may,approve,'.'],[^,'If',a,n,:,licensor,commissions,the,licensee,then,a,licensee,has,to,comment,'.'],[^,'If',the,n,:,licensor,does,not,approve,then,the,licensee,may,not,comment,'.'],[^,'If',the,licensee,may,not,publish,then,the,comments,have,to,be,removed,within,24,hours,'.'],[^,'If',the,comments,have,to,be,removed,and,are,not,removed,within

In [28]:
/home/jovyan/bin/ape.bin -file license_from_prolog.ace -solo paraphrase

There is an arbiter X1.
The arbiter X1 is a person.

There is a n:licensor X1.
The n:licensor X1 is a person.

There is a licensee X1.
The licensee X1 is a person.

There is a n:sublicensee X1.
The n:sublicensee X1 is a person.

There is a n:licensor X1.
The n:licensor X1 commissions a licensee.

There is a n:duration-to-remove-comments X1.
The n:duration-to-remove-comments X1 is 24 hours.

There is a n:license-fee X1.
The n:license-fee X1 is an amount.

If there is a n:licensor and there is a licensee then a n:license-fee is required.

If a n:license-fee is paid then a licensee receives a license.

If there is a license then there is a licensee X1 and it is possible that the licensee X1 v:sublicense .

If there is a license then there is a licensee X1 and it is possible that the licensee X1 comments.

If there is a licensee X1 and it is admissible that the licensee X1 comments then there is a n:licensor X2 and it is admissible that the n:licensor X2 approves.

If a n:licensor commissi